SqueezeNet: AlexNet-level accuracy with 50x fewer parameters and <0.5MB model size: https://arxiv.org/pdf/1602.07360.pdf

In [1]:
from tensorflow import keras

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


In [5]:
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout, GlobalAveragePooling2D, BatchNormalization

In [6]:
def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    x = Convolution2D(squeeze, (1, 1), padding='valid', name=s_id + 'squeeze1x1')(x)
    x = Activation('relu', name=s_id + 'relu_squeeze1x1')(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + 'expand1x1')(x)
    left = Activation('relu', name=s_id + 'relu_expand1x1')(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + 'expand3x3')(x)
    right = Activation('relu', name=s_id + 'relu_expand3x3')(right)

    x = concatenate([left, right], axis=3, name=s_id + 'concat')

    return x


In [7]:
# define input
input_img = Input(shape=(32, 32, 3))
num_classes=10

# define the network
x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(input_img)
x = Activation('relu', name='relu_conv1')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

x = fire_module(x, fire_id=2, squeeze=16, expand=64)
x = fire_module(x, fire_id=3, squeeze=16, expand=64)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

x = fire_module(x, fire_id=4, squeeze=32, expand=128)
x = fire_module(x, fire_id=5, squeeze=32, expand=128)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

x = fire_module(x, fire_id=6, squeeze=48, expand=192)
x = fire_module(x, fire_id=7, squeeze=48, expand=192)
x = fire_module(x, fire_id=8, squeeze=64, expand=256)
x = fire_module(x, fire_id=9, squeeze=64, expand=256)

x = Dropout(0.5, name='drop9')(x)

x = Convolution2D(num_classes, (1, 1), padding='valid', name='conv10')(x)
x = Activation('relu', name='relu_conv10')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax', name='loss')(x)

# create the model
model = Model(inputs=input_img, outputs=x, name='squeezenet')

In [11]:
opt = keras.optimizers.Adam(learning_rate=1e-4, clipvalue=0.5)
model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer=opt, metrics=['accuracy',])

In [12]:
history = model.fit(x_train, y_train, batch_size=32, epochs=1)

1563/1563 [==============================] - 99s 62ms/step - loss: 1.9716 - accuracy: 0.2257
